In [ ]:
from owslib.wmts import WebMapTileService
import pandas as pd
from geopy.distance import great_circle
import matplotlib.pyplot as plt
import pyproj
from IPython.display import Image, display
import math
import time


## Experimental Testing for getting photo's out of multiple WMTS servers.
This Notebook experiments with a way  to get photo's as a datasource to later integrate it into the SailingRobots Website. 

The static variables like the website and the normal map layer.

In [ ]:
try:
    wmts_avoin = WebMapTileService('https://avoin-karttakuva.maanmittauslaitos.fi/avoin/wmts')
    time.sleep(0.01)
    wmts_maasto = WebMapTileService('https://karttamoottori.maanmittauslaitos.fi/maasto/wmts')
except Exception as e:
    print(e)
tile_matrix_set_name = 'ETRS-TM35FIN'
map_layer = 'ortokuva'
standardized_rendering_pixel_size = 0.00028


The webmap tile service can be used for getting the images and calculating the coordinates of them

In [ ]:
def get_info_wmts(wmts):
    print('possible maps:', list(wmts.contents.keys()))
    print('possible coordinate system:', list(wmts.tilematrixsets.keys()))
    print('possible formats :', list(wmts.contents[map_layer].formats))
    print('length of the formats', len(wmts.tilematrixsets[tile_matrix_set_name].tilematrix))


get_info_wmts(wmts_avoin)


The data from the xml that cannot be parsed yet
```xml
<ows:LowerCorner>-548576.000000 6291456.000000</ows:LowerCorner>
<ows:UpperCorner>1548576.000000 8388608.000000</ows:UpperCorner>
```
it is in the following format <lon,lat>
```python
   #ll = LowerCorner
   #ur = UpperCorner
 self.boundingBoxWGS84 = (ll[0], ll[1], ur[0], ur[1])
 ```


In [ ]:
def convert_coordinate_systems(lat, lon, inverse=False, src='epsg:3067', destination='epsg:4326'):
    """Converts Coordinate System to a different System.
    Default From Finnish System(ETRS-TM35FIN) to WGS84. If inverse is passed then they are swapped around.
    """
    if inverse:
        src, destination = destination, src
    proj_src = pyproj.Proj(init=src)
    proj_dest = pyproj.Proj(init=destination)
    transformed = pyproj.transform(proj_src, proj_dest, lon, lat)
    return transformed


In [ ]:
def calculate_distance(lat, lon, lat1, lon1):
    lat_lon = (lat, lon)
    lat_lon1 = (lat1, lon1)
    return great_circle(lat_lon, lat_lon1)


def distance_meter(lat, lon, lat1, lon1):
    return calculate_distance(lat, lon, lat1, lon1).m


def distance_kilometer(lat, lon, lat1, lon1):
    return calculate_distance(lat, lon, lat1, lon1).km


def calculate_distance_finnish(lat, lon, lat1, lon1, distance_type='km'):
    get_ = convert_coordinate_systems(lat, lon)
    get_1 = convert_coordinate_systems(lat1, lon1)
    if distance_type == 'm':
        distance_func = distance_meter
    else:
        distance_func = distance_kilometer
    return distance_func(get_[1], get_[0], get_1[1], get_1[0])


In [ ]:
def get_width_height_covered_by_matrix(matrix, distance_type=None):
    delta_width = matrix.tilewidth * standardized_rendering_pixel_size * matrix.scaledenominator
    delta_height = matrix.tileheight * standardized_rendering_pixel_size * matrix.scaledenominator
    lon1 = matrix.topleftcorner[0] + delta_width
    lat1 = matrix.topleftcorner[1] - delta_height
    distance = calculate_distance_finnish(lon=matrix.topleftcorner[0],
                                          lat=matrix.topleftcorner[1],
                                          lon1=lon1,
                                          lat1=lat1, distance_type=distance_type)
    return distance


In [ ]:
def get_boxes_by__matrix(matrix):
    delta_width = matrix.tilewidth * standardized_rendering_pixel_size * matrix.scaledenominator
    delta_height = matrix.tileheight * standardized_rendering_pixel_size * matrix.scaledenominator
    boxes_dict = dict()
    for tile_height_pos in range(matrix.matrixheight):
        for tile_width_pos in range(matrix.matrixwidth):
            left_lon = matrix.topleftcorner[0] + (tile_width_pos * delta_width)
            left_lat = matrix.topleftcorner[1] - (tile_height_pos * delta_height)
            right_lon = matrix.topleftcorner[0] + ((tile_width_pos + 1) * delta_width)
            right_lat = matrix.topleftcorner[1] - ((tile_height_pos + 1) * delta_height)
            left = convert_coordinate_systems(lat=left_lat, lon=left_lon)
            right = convert_coordinate_systems(right_lat, right_lon)

            combined = dict(left=left, right=right)
            boxes_dict[tile_height_pos, tile_width_pos] = combined
    return boxes_dict


def get_single_by_matrix(matrix, lat, lon):
    delta_width = matrix.tilewidth * standardized_rendering_pixel_size * matrix.scaledenominator
    delta_height = matrix.tileheight * standardized_rendering_pixel_size * matrix.scaledenominator
    finn_coordinates = convert_coordinate_systems(lat=lat, lon=lon, inverse=True)

    width = math.trunc((finn_coordinates[0] - matrix.topleftcorner[0]) / delta_width)
    height = math.trunc((matrix.topleftcorner[1] - finn_coordinates[1]) / delta_height)
    return height, width


def get_boxes_by_df(matrix):
    boxes = get_boxes_by__matrix(matrix)
    df = pd.DataFrame(boxes).T
    df_ = pd.concat([df['left'].apply(pd.Series), df['right'].apply(pd.Series)], keys=['left', 'right'], axis=1)
    return df_


In [ ]:
def plot_rectangles(df):
    fig, ax = plt.subplots(1, 1)
    color_items = ['blue', 'yellow', 'red', 'green']
    count = 0
    for x in df.values:
        if count == len(color_items):
            count = 0
        rect = plt.Rectangle((x[0], x[1]), x[2] - x[0], x[3] - x[1], color=color_items[count])
        count += 1
        ax.add_patch(rect)
    ax.autoscale_view()


1 is N/Latitude 0 is E/Longitude

In [ ]:
def get_column_row_for_coordinate_at_level(latitude, longitude, wmts_, tileset=None, level=2, df=None):
    if tileset == None and df == None:
        tileset = wmts_.tilematrixsets[tile_matrix_set_name]
        list_of_tilematrixes = list(tileset.tilematrix.keys())
        df = get_boxes_by_df(tileset.tilematrix[list_of_tilematrixes[level]])
    searched_df = df[(df['left'][0] <= longitude)
                     & (longitude <= df['right'][0])
                     & (df['left'][1] >= latitude)
                     & (latitude >= df['right'][1])]
    index_ = searched_df.index
    if len(index_.to_list()) == 0:
        print(index_)
        display(df)
        display(searched_df)

        return
    codes = index_.to_list()[-1]
    return codes, df


def get_column_row_for_coordinate_at_level_single(latitude, longitude, wmts, tileset=None, level=2):
    if tileset == None:
        tileset = wmts.tilematrixsets[tile_matrix_set_name]
        list_of_tilematrixes = list(tileset.tilematrix.keys())

        codes = get_single_by_matrix(tileset.tilematrix[list_of_tilematrixes[level]], lat=latitude, lon=longitude)
        return codes


In [ ]:
def get_image_for_coordinate(latitude, longitude, level, wmts_, specified_map_layer=map_layer):
    codes = get_column_row_for_coordinate_at_level_single(latitude=latitude,
                                                          longitude=longitude,
                                                          wmts=wmts_,
                                                          level=level)
    tile = wmts_.gettile(layer=specified_map_layer,
                         tilematrixset=tile_matrix_set_name,
                         tilematrix='{0}'.format(level),
                         row=codes[0],
                         column=codes[1],
                         format="image/jpeg")
    out = open('test.jpg', 'wb')
    out.write(tile.read())
    out.close()
    display(Image(filename='test.jpg'))


In [ ]:
get_image_for_coordinate(60.062936, 19.968358, level=15, wmts_=wmts_avoin)
get_image_for_coordinate(60.062936, 19.968358, level=15, wmts_=wmts_maasto)
get_image_for_coordinate(60.062936, 19.968358, level=15, wmts_=wmts_maasto, specified_map_layer='ortokuva_vaaravari')
